In this script we have done descriptive analysis of the billing consumption data, Analysis include stats such as KWH consumption per day, Current charges per day. Each of the code block will have brief description.

In below code block we importing libraries required for processing and looading data in to python.In "pd.read_csv" we need to provide the exact system path of the file that needs to be processed.

In [462]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]
#read consumption file
billing_period = pd.read_csv(r"C:\Users\ijohan\Desktop\the-spoon\projects-code\power_sector\JOINED_BILLING_SPOTDATA_MAY2019_FOUNDATION_V3.csv",sep='|', encoding='iso-8859-1')

Below code block is to view the structure of data(first few rows)

In [3]:
billing_period.head()

In below code block we are droping all the columns where all the values are NA

In [464]:
# drop columns having all NAN values
billing_period = billing_period.dropna(axis = 1, how = 'all')

Below we are converting all the columns with date as date data type

In [465]:
# convert date and time field to date
billing_period['Bill date'] = pd.to_datetime(billing_period['Bill date']).apply(lambda x: x.date())
billing_period['Billing period start date'] = pd.to_datetime(billing_period['Billing period start date']).apply(lambda x: x.date())
billing_period['Billing period end date'] = pd.to_datetime(billing_period['Billing period end date']).apply(lambda x: x.date())

In [466]:
# Check the column names of the file 
billing_period.columns

Index(['Contract Account', 'Portion', 'Bill date', 'Bill type',
       'Billing period start date', 'Billing period end date', 'Bill duration',
       'Prorate factor', 'MRU', 'Description', 'BILLING_PERIOD',
       'CUSTOMER_NAME', 'AREA', 'READING_TYPE', 'TARIFTYP',
       'ACC_DETERMINATION', 'OUTSTANDING_AMOUNT', 'CURRENT_CHARGES',
       'ROUNDING_AMOUNT', 'TOTAL_BILL', 'KWH TOTAL', 'KWH_RM_1', 'KWH_RM_2',
       'KWH_RM'],
      dtype='object')

In [467]:

# While reading the file to python, sometime it reads lasrge numbers as scientific notation.
# so , we are converting CA field from scientific notation to string
billing_period['Contract Account'] = billing_period['Contract Account'].map(lambda x: '{:.0f}'.format(x))

In [468]:
# Converting CA field of billing_period to string
billing_period['Contract Account'] = billing_period['Contract Account'].astype(str)

In below code block we are filtering only required columns

In [471]:
# filtering required columns
billing_period = billing_period[['Contract Account', 'Portion', 'Bill type',
       'Bill duration', 'Prorate factor', 'MRU', 'Description',
       'BILLING_PERIOD', 'AREA', 'READING_TYPE', 'TARIFTYP',
       'ACC_DETERMINATION', 'OUTSTANDING_AMOUNT', 'CURRENT_CHARGES',
      'TOTAL_BILL', 'KWH TOTAL', 'KWH_RM']]

In [472]:
#In this code block we filtering data based on year. In this case we take data for 2019 or all the data other than 2018
billing_period = billing_period[billing_period.BILLING_PERIOD != 2018]

In [474]:
# Removing rows where bill duration is NAN
billing_period = billing_period[np.isfinite(billing_period['Bill duration'])]

In [4]:
billing_period.head()

In [476]:
# Check data types of "billing_period" data frame
billing_period.dtypes

# KWH per day

In below code block we are trying to find the KWH consumption per day based on CA. We are creating new column for KWH per day

In [477]:
# create new column for KWH per day consumption in month for each CA
billing_period['KWH_PER_DAY'] = billing_period.apply(lambda x: x['Bill duration'] if x['Bill duration'] < 1 else x['KWH TOTAL']/x['Bill duration'], axis=1)

In [5]:
# Check if the new column is added to the data frame
billing_period.head()

In below code block we are trying to find the billing charges per day based on CA. We are creating new column for CHARGES_PER_DAY

# Billing charges per day

In [479]:
# create new column for CURRENT_CHARGES per day consumption in month for each CA
billing_period['CHARGES_PER_DAY'] = billing_period.apply(lambda x: x['Bill duration'] if x['Bill duration'] < 1 else x['CURRENT_CHARGES']/x['Bill duration'], axis=1)

In [6]:
#check if the CHARGES_PER_DAY column is added to the data frame
billing_period.head()

In [481]:
# Writing the file as csv to form required output.
billing_period.to_csv("C:\\MPS\\projects\\TNB\\billing\\data\\billing_mode_data.csv")

# Current charges per KWH: CCPerKWH

In below code block we are trying to find the current charges per KWH based on CA. We are creating new column for CURRENT_CHARGES_PER_KWH

In [482]:
# create new column for CURRENT_CHARGES per day consumption in month for each CA
billing_period['CURRENT_CHARGES_PER_KWH'] = billing_period.apply(lambda x: x['CHARGES_PER_DAY'] if x['CHARGES_PER_DAY'] < 1 else x['KWH_PER_DAY']/x['CHARGES_PER_DAY'], axis=1)

In [7]:
# Check if the CURRENT_CHARGES_PER_KWH column is added to data frame
billing_period.head()

# Cumulative sum of KWH per day: CumSumKWHperDay

In below code block we are trying to find the Cumulative sum of KWH per day based on CA. We are creating new column for cum_sum_KWH_per_day

In [484]:
# Find cumulative sum of each CA
billing_period['cum_sum_KWH_per_day'] = (billing_period['KWH_PER_DAY']).groupby(billing_period['Contract Account']).cumsum()

In [9]:
# Check if cum_sum_KWH_per_day column is added to the data to data frame
billing_period.head()

# Cum num column

In [486]:
# Adding new column numbering groupby CA column, number to days corresponding to cumulative sum
billing_period['cum_num'] = billing_period.groupby(['Contract Account']).cumcount()+1

In [487]:
billing_period.head()

,Contract Account,Portion,Bill type,Bill duration,Prorate factor,MRU,Description,BILLING_PERIOD,AREA,READING_TYPE,...,OUTSTANDING_AMOUNT,CURRENT_CHARGES,TOTAL_BILL,KWH TOTAL,KWH_RM,KWH_PER_DAY,CHARGES_PER_DAY,CURRENT_CHARGES_PER_KWH,cum_sum_KWH_per_day,cum_num
0,220225000000,SPOT28,OPC,30.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/01,6121,Bacaan Sebenar,...,0.00,34.23,34.25,157.0,34.23,5.233333,1.141000,4.586620,5.233333,1
1,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/02,6121,Bacaan Sebenar,...,0.00,27.69,27.70,127.0,27.69,4.096774,0.893226,0.893226,9.330108,2
2,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/03,6121,Bacaan Sebenar,...,0.00,11.55,11.55,53.0,11.55,1.709677,0.372581,0.372581,11.039785,3
3,220225000000,SPOT28,OPC,18.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/04,6121,Bacaan Akhir,...,11.55,-537.88,-526.34,17.0,3.71,0.944444,-29.882222,-29.882222,11.984229,4
4,210000047810,SPOT21,OPC,30.0,1.0,16000563.0,B 383 SEK 17 SHAH ALAM,2019/01,6160,Bacaan Sebenar,...,0.00,261.35,261.35,644.0,255.82,21.466667,8.711667,2.464129,21.466667,1


# Cumulative average

In [488]:
# cumulative KWH per day/ cum_num
# create new column for CURRENT_CHARGES per day consumption in month for each CA
billing_period['CUM_KWH_AVG'] = billing_period.apply(lambda x: x['cum_num'] if x['cum_num'] < 1 else x['cum_sum_KWH_per_day']/x['cum_num'], axis=1)

In [489]:
billing_period.head()

,Contract Account,Portion,Bill type,Bill duration,Prorate factor,MRU,Description,BILLING_PERIOD,AREA,READING_TYPE,...,CURRENT_CHARGES,TOTAL_BILL,KWH TOTAL,KWH_RM,KWH_PER_DAY,CHARGES_PER_DAY,CURRENT_CHARGES_PER_KWH,cum_sum_KWH_per_day,cum_num,CUM_KWH_AVG
0,220225000000,SPOT28,OPC,30.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/01,6121,Bacaan Sebenar,...,34.23,34.25,157.0,34.23,5.233333,1.141000,4.586620,5.233333,1,5.233333
1,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/02,6121,Bacaan Sebenar,...,27.69,27.70,127.0,27.69,4.096774,0.893226,0.893226,9.330108,2,4.665054
2,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/03,6121,Bacaan Sebenar,...,11.55,11.55,53.0,11.55,1.709677,0.372581,0.372581,11.039785,3,3.679928
3,220225000000,SPOT28,OPC,18.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/04,6121,Bacaan Akhir,...,-537.88,-526.34,17.0,3.71,0.944444,-29.882222,-29.882222,11.984229,4,2.996057
4,210000047810,SPOT21,OPC,30.0,1.0,16000563.0,B 383 SEK 17 SHAH ALAM,2019/01,6160,Bacaan Sebenar,...,261.35,261.35,644.0,255.82,21.466667,8.711667,2.464129,21.466667,1,21.466667


In [490]:
#billing_period.to_csv("C:\\MPS\\projects\\TNB\\billing\\data\\billing_mode_data_avg.csv")

# Converting estimated and actual to english

In [491]:
# We replacing the column labels from Malay to english in reading type column
billing_period['READING_TYPE'] = billing_period['READING_TYPE'].replace(['Bacaan Sebenar'], 'ACT')
billing_period['READING_TYPE'] = billing_period['READING_TYPE'].replace(['Bacaan Anggaran'], 'EST')
billing_period['READING_TYPE'] = billing_period['READING_TYPE'].replace(['Bacaan Akhir'], 'FIN')

# Create billing_period_flag for billling data

In below code block we are creatign flag coresponding to each CA based upon the Billing duration

In [492]:
#Creating flag column with below conditions for billing period
def billing_period_flag(row):
    if row['Bill duration'] < 30:
        val = 'less than 30'
    elif (row['Bill duration'] == 30) | (row['Bill duration'] == 31):
        val = '30 or 31'
    elif (row['Bill duration'] > 31) & (row['Bill duration'] < 40):
        val = "between 31 and 40"
    elif row['Bill duration'] >= 40:
        val = 'greater than 40'
    return val

In [493]:
#applying fuction on billing period data
billing_period['billing_period_flag'] = billing_period.apply(billing_period_flag, axis=1)

In [494]:
billing_period.head()

,Contract Account,Portion,Bill type,Bill duration,Prorate factor,MRU,Description,BILLING_PERIOD,AREA,READING_TYPE,...,TOTAL_BILL,KWH TOTAL,KWH_RM,KWH_PER_DAY,CHARGES_PER_DAY,CURRENT_CHARGES_PER_KWH,cum_sum_KWH_per_day,cum_num,CUM_KWH_AVG,billing_period_flag
0,220225000000,SPOT28,OPC,30.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/01,6121,ACT,...,34.25,157.0,34.23,5.233333,1.141000,4.586620,5.233333,1,5.233333,30 or 31
1,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/02,6121,ACT,...,27.70,127.0,27.69,4.096774,0.893226,0.893226,9.330108,2,4.665054,30 or 31
2,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/03,6121,ACT,...,11.55,53.0,11.55,1.709677,0.372581,0.372581,11.039785,3,3.679928,30 or 31
3,220225000000,SPOT28,OPC,18.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/04,6121,FIN,...,-526.34,17.0,3.71,0.944444,-29.882222,-29.882222,11.984229,4,2.996057,less than 30
4,210000047810,SPOT21,OPC,30.0,1.0,16000563.0,B 383 SEK 17 SHAH ALAM,2019/01,6160,ACT,...,261.35,644.0,255.82,21.466667,8.711667,2.464129,21.466667,1,21.466667,30 or 31


# Create spot_period_flag for billling data

In below code block we are creating flag with spot period based on the spot_period_flag column that we have derived above

In [496]:
billing_period['Portion'] = billing_period['Portion'].astype(str)

In [497]:
# Create a new column as a flag for spot days

def spot_period_flag(row):
    if (row['Portion'][4:] == '17') | (row['Portion'][4:] == '18'):
        val = 'Spot btw 17 to 20'
    elif (row['Portion'][4:] == '19') | (row['Portion'][4:] == '20'):
        val = 'Spot btw 17 to 20'
    else:
        val = 'others'
    return val

In [498]:
#applying fuction on for spot period flag on billing data
billing_period['spot_period_flag'] = billing_period.apply(spot_period_flag, axis=1)

In [499]:
billing_period.head()

,Contract Account,Portion,Bill type,Bill duration,Prorate factor,MRU,Description,BILLING_PERIOD,AREA,READING_TYPE,...,KWH TOTAL,KWH_RM,KWH_PER_DAY,CHARGES_PER_DAY,CURRENT_CHARGES_PER_KWH,cum_sum_KWH_per_day,cum_num,CUM_KWH_AVG,billing_period_flag,spot_period_flag
0,220225000000,SPOT28,OPC,30.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/01,6121,ACT,...,157.0,34.23,5.233333,1.141000,4.586620,5.233333,1,5.233333,30 or 31,others
1,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/02,6121,ACT,...,127.0,27.69,4.096774,0.893226,0.893226,9.330108,2,4.665054,30 or 31,others
2,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/03,6121,ACT,...,53.0,11.55,1.709677,0.372581,0.372581,11.039785,3,3.679928,30 or 31,others
3,220225000000,SPOT28,OPC,18.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/04,6121,FIN,...,17.0,3.71,0.944444,-29.882222,-29.882222,11.984229,4,2.996057,less than 30,others
4,210000047810,SPOT21,OPC,30.0,1.0,16000563.0,B 383 SEK 17 SHAH ALAM,2019/01,6160,ACT,...,644.0,255.82,21.466667,8.711667,2.464129,21.466667,1,21.466667,30 or 31,others


# Flag for tarif change


In below code block we are creating new flag column for the CA based on the Tarif change

In [504]:

def tarif_change_flag(row):
    if (row['KWH TOTAL'].shift() < 300) & (row['KWH TOTAL'].shift(-1) > 300):
        val = 'tarif change'
    else:
        val = 'others'
    return val


In [505]:
billing_period = billing_period.fillna(0)
billing_period['KWH TOTAL'] = billing_period['KWH TOTAL'].astype(int)
billing_period = billing_period.reset_index(drop=True)

In [507]:
cond1 = billing_period['KWH TOTAL'] > 300 

billing_period['tarif_change_flag'] = np.where(cond1, 1, 0)

In [513]:
billing_period['tarif_change_flag1'] = billing_period.groupby('Contract Account')['tarif_change_flag'].diff().abs()

In [514]:
billing_period['tarif_change_flag2'] = billing_period['tarif_change_flag1'].map({1:'tarif change', 0:'normal'}) 

In [515]:
del billing_period['tarif_change_flag1']
del billing_period['tarif_change_flag']

In [516]:
billing_period.head()

,Contract Account,Portion,Bill type,Bill duration,Prorate factor,MRU,Description,BILLING_PERIOD,AREA,READING_TYPE,...,KWH_RM,KWH_PER_DAY,CHARGES_PER_DAY,CURRENT_CHARGES_PER_KWH,cum_sum_KWH_per_day,cum_num,CUM_KWH_AVG,billing_period_flag,spot_period_flag,tarif_change_flag2
0,220225000000,SPOT28,OPC,30.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/01,6121,ACT,...,34.23,5.233333,1.141000,4.586620,5.233333,1,5.233333,30 or 31,others,NaN
1,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/02,6121,ACT,...,27.69,4.096774,0.893226,0.893226,9.330108,2,4.665054,30 or 31,others,normal
2,220225000000,SPOT28,OPC,31.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/03,6121,ACT,...,11.55,1.709677,0.372581,0.372581,11.039785,3,3.679928,30 or 31,others,normal
3,220225000000,SPOT28,OPC,18.0,1.0,12100820.0,"A 4467 BLK A-G-01, JLN PJU 8/2",2019/04,6121,FIN,...,3.71,0.944444,-29.882222,-29.882222,11.984229,4,2.996057,less than 30,others,normal
4,210000047810,SPOT21,OPC,30.0,1.0,16000563.0,B 383 SEK 17 SHAH ALAM,2019/01,6160,ACT,...,255.82,21.466667,8.711667,2.464129,21.466667,1,21.466667,30 or 31,others,NaN


In [517]:
billing_period.to_csv(r"C:\Users\ijohan\Desktop\the-spoon\projects-code\power_sector\billing_consumption_second_analysis_tarif_V3.csv")

# Word count frequency 

In below code block we are counting the word frequesncy based in the SN_Remarks columns to come up with the possible keywords for RCA

In [5]:
#SN_Remarks_Working_File_18062019.xlsx
import pandas as pd
keywords = pd.read_excel(r"C:\Users\ijohan\Desktop\the-spoon\projects-code\power_sector\SN_Remarks_Working_File_18062019.xlsx")

In [6]:
keywords.head()

,Service_Request_ID,CA_Number,SN_Number,SO_Number,SN_Remark,Unnamed: 5
0,1000632215,210113162600,218000599947,111000654303,Internal note03.05.2019 09:50:32 60068167Pengg...,Completed
1,1000638648,220030804400,218000601324,111000655723,Internal note07.05.2019 10:42:58 60068143pengg...,Completed
2,1000644697,220294238500,218000603800,111000658527,Internal note13.05.2019 13:36:35 60068120pengg...,Completed
3,1000655580,220013207406,210000006171,112000115493,Internal note 29.05.2019 13:24:19 ...,Completed
4,1000655784,220005842708,201000159284,113000767661,Internal note 24.05.2019 22:01:12 60068082 pe...,Completed


In [7]:
#k_count = keywords['SN_Remark'].str.split(expand=True).stack().value_counts()
k_count = (keywords['SN_Remark'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('vals')
              .reset_index(name='count'))

In [11]:
#k_count.head(20)

In [10]:
len(k_count)

34500

In [12]:
# generate unigrams 
i = keywords.SN_Remark\
      .str.lower()\
      .str.replace('[^a-z\s]', '')\
      .str.split(expand=True)\
      .stack()

In [13]:
# generate bigrams by concatenating unigram columns
j = i + ' ' + i.shift(-1)
# generate trigrams by concatenating unigram and bigram columns
k = j + ' ' + i.shift(-2)

In [14]:
# concatenate all series vertically, and remove NaNs
k_grams = pd.concat([i, j, k]).dropna().reset_index(drop=True)

In [16]:
k_grams.head(20)

0       internal
1           note
2       pengguna
3         maklum
4            bil
5           pada
6      terlampau
7         tinggi
8       sehinnga
9        cecahrm
10         dalam
11        sistem
12    penggunaan
13        beliau
14      hanyalah
15         lebih
16        kurang
17            rm
18       kebawah
19      tiaptiap
dtype: object

In [33]:
k_grams = pd.DataFrame(k_grams, columns=['words'])

In [44]:
counts = k_grams['words'].value_counts().to_dict()

In [47]:
k_grams_count = pd.DataFrame(counts.items(), columns=['words', 'count'])

In [51]:
k_grams_count.head()

,words,count
0,pengguna,6856
1,bil,4526
2,mohon,3877
3,note,3853
4,internal,3836


In [52]:
k_grams_count.to_csv(r"C:\Users\ijohan\Desktop\the-spoon\projects-code\power_sector\keywords_grams.csv")